In [1]:
import tensorflow
import numpy as np
import keras
import os
from utility_functions import load_dataset

path_to_file = '/home/alexandr/data/audio/'
path_to_file_test = '/home/alexandr/data/test/'
path_to_file_unknown = '/home/alexandr/data/unknown/'

Using TensorFlow backend.


In [2]:
# extract classes from txt file

raw_classes = np.loadtxt('/home/alexandr/data/meta/meta.txt', dtype='str')
#print(raw_classes)
classes = np.delete(raw_classes, [1, 2, 3], 1)
text_labels = ['background', 'bags', 'door', 'keyboard', 'knocking_door', 'ring', 'speech', 'tool']

for index, label in enumerate(text_labels):
    classes[classes == label] = index

dict_classes = dict(classes)
classes_test = np.loadtxt('./meta-test.txt',dtype='str')
dict_classes_test = dict(classes_test)

raw_data_train = np.array(load_dataset(path_to_file, dict_classes))
raw_data_test = np.array(load_dataset(path_to_file_test, dict_classes_test))
raw_data_unknown = np.array(load_dataset(path_to_file_unknown))

In [3]:
from detector import detect_startpoint
# 
# cropped_data_train = []
# for sound in raw_data_train[:,0]:
#     start_point = detect_startpoint(sound)
#     cropped_data_train.append(sound[start_point:])
# cropped_data_train = np.array(cropped_data_train)
# print (1)
# cropped_data_test = []
# for sound in raw_data_test[:,0]:
#     start_point = detect_startpoint(sound)
#     cropped_data_test.append(sound[start_point:])
# cropped_data_test = np.array(cropped_data_test)
cropped_data_train =raw_data_train
cropped_data_test = raw_data_test
cropped_data_unknown = raw_data_unknown

In [4]:
# first second segmentation
from utility_functions import find_longest, padding_data, get_random_segment

# max_length = find_longest(raw_data)
# pad_data = padding_data(raw_data, max_length)
segmented_data_train = []

for index, sound in enumerate(cropped_data_train[:,0]):
    segmented_data_train.append(get_random_segment(sound))
    
segmented_data_train = np.array(segmented_data_train)


segmented_data_test= []

for index, sound in enumerate(cropped_data_test[:,0]):
    segmented_data_test.append(get_random_segment(sound))
    
segmented_data_test = np.array(segmented_data_test)

segmented_data_unknown = []

for index, sound in enumerate(cropped_data_unknown):
    segmented_data_unknown.append(get_random_segment(sound))

In [5]:
# clean memory
pad_data = None
raw_data = None
extracted_features = None

In [7]:
print(np.array(segmented_data_train).shape)
print(np.array(segmented_data_test).shape)
print(np.array(raw_data_train).shape)
print(np.array(raw_data_test).shape)
print(np.array(segmented_data_unknown).shape)


(11307, 48000)
(473, 48000)
(11307, 2)
(473, 2)
(137, 48000)


In [6]:
import librosa
from speechpy.feature import lmfe

extracted_features_train = []
extracted_features_test = []
extracted_features_unknown = []

def extract_feature(mode='mfcc'):
    if mode == 'mfcc':
        for sound in segmented_data_train:
            feature = librosa.feature.mfcc(sound.astype(float), sr=16000, n_mfcc=20)
            extracted_features_train.append(feature)

        for sound in segmented_data_test:
            extracted_features_test.append(librosa.feature.mfcc(sound.astype(float), sr=16000, n_mfcc=20))
    elif mode == 'mbe':

        for sound in segmented_data_train:
            feature = lmfe(sound.astype(float), 16000)
            extracted_features_train.append(feature)

        for sound in segmented_data_test:
            extracted_features_test.append(lmfe(sound.astype(float), 16000))
            
        for sound in segmented_data_unknown:
            extracted_features_unknown.append(lmfe(sound.astype(float), 16000))    
            
extract_feature('mbe')
x_train = np.array(extracted_features_train)
x_test = np.array(extracted_features_test)
x_unknown = np.array(extracted_features_unknown)


In [8]:
# temporary cell
print(x_train.shape)
y_train = np.array(raw_data_train[:,1])
print(y_train.shape)
print(x_test.shape)
y_test = np.array(raw_data_test[:,1])
print(y_test.shape)

print(x_unknown.shape)


(11307, 298, 40)
(11307,)
(473, 298, 40)
(473,)
(137, 298, 40)


In [5]:
raw_data_test[:,1].shape

(473,)

In [9]:
filename = 'mbe_3sec_random_default'
np.save('/home/alexandr/'+ filename, x_train)
np.save('/home/alexandr/y', y_train)
np.save('/home/alexandr/' + filename+ '_test', x_test)
np.save('/home/alexandr/y_test', y_test)
np.save('/home/alexandr/x_unknown',x_unknown)

In [7]:
y_test = np.array(raw_data_test[:,1])

np.save('/home/alexandr/y_test', y_test)